In [1]:
import xarray as xr
import os
import numpy as np

In [2]:
grid_dir = '/home/disk/sipn/nicway/data/grids'

In [3]:
ds_region = xr.open_dataset(os.path.join(grid_dir, 'sio_2016_mask.nc'))
ds_region.set_coords(['lat','lon'], inplace=True);
# Note minor -0.000004 degree differences in latitude
# Replace with nsdidc obs file lat/lon
ds_nsidc = xr.open_dataset('/home/disk/sipn/nicway/data/obs/NSIDC_0081/sipn_nc/nt_20170804_f18_nrt_n.nc')
ds_region['lat'] = ds_nsidc.lat
ds_region['lon'] = ds_nsidc.lon
# Clean up string (need to do at import)
ds_region.coords['region_names'] = xr.DataArray([x.item(0).decode("utf-8").strip() for x in ds_region.region_names],
                                                dims='nregions', coords={'nregions':ds_region.nregions})
ds_region.rename({'nx':'x', 'ny':'y'}, inplace=True);
ds_region['nregions'] = np.arange(1,22)
rm_regs = ['Ice-free Oceans', 'null', 'land outline', 'land' ]
# Get regions index for ones we want
ds_region.coords['ocean_regions']  = xr.DataArray(ds_region.where(~ds_region.region_names.isin(rm_regs), drop=True).nregions.values,
                                                dims='ocean_regions')
ds_region.area.attrs = '' # was complaining about coords in attrs so remove
# drop time
ds_region = ds_region.drop('time')
ds_region.to_netcdf(os.path.join(grid_dir,'sio_2016_mask_Update.nc'))

In [4]:
ds_region

<xarray.Dataset>
Dimensions:        (nregions: 21, ocean_regions: 14, x: 304, y: 448)
Coordinates:
    lat            (x, y) float64 ...
    lon            (x, y) float64 ...
    region_names   (nregions) <U17 'Ice-free Oceans' 'Sea of Okhotsk' ...
  * x              (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * y              (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    hole_mask      (y, x) bool ...
    xm             (x) int64 ...
    ym             (y) int64 ...
  * nregions       (nregions) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
  * ocean_regions  (ocean_regions) int64 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Data variables:
    mask           (x, y) int32 ...
    area           (x, y) float32 ...